In [1]:
import pandas as pd
from pathlib import Path

df = pd.read_csv('datasets/synthetic_logs.csv')
df.head()

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\tamil\AppData\Local\Temp\ipykernel_24724\519163348.py:4: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('training\datasets\synthetic_logs.csv')
C:\Users\tamil\AppData\Local\Temp\ipykernel_24724\519163348.py:4: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('training\datasets\synthetic_logs.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'training\\datasets\\synthetic_logs.csv'

In [9]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [13]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [14]:
df[df.target_label=='HTTP Status'].sample(10)

,timestamp,source,log_message,target_label,complexity
818,2025-11-16 10:43:20,ThirdPartyAPI,nova.osapi_compute.wsgi.server [req-1c33bdb8-a...,HTTP Status,bert
706,2025-06-16 02:09:56,ThirdPartyAPI,nova.metadata.wsgi.server [req-5eee7c2e-9d80-4...,HTTP Status,bert
1965,2025-11-30 17:05:07,ModernHR,nova.metadata.wsgi.server [req-2d24fed7-3524-4...,HTTP Status,bert
2314,2025-08-19 23:28:47,ThirdPartyAPI,nova.osapi_compute.wsgi.server [req-dd5eea8f-e...,HTTP Status,bert
1583,2025-04-20 15:06:21,ModernCRM,nova.osapi_compute.wsgi.server [req-428f28af-b...,HTTP Status,bert
1357,2025-02-11 22:11:07,AnalyticsEngine,nova.osapi_compute.wsgi.server [req-b066b9b3-3...,HTTP Status,bert
1605,2025-06-16 05:56:34,BillingSystem,nova.osapi_compute.wsgi.server [req-a2cd93b2-1...,HTTP Status,bert
1366,2025-06-05 21:26:32,ModernHR,nova.metadata.wsgi.server [req-a189d358-853f-4...,HTTP Status,bert
909,2025-03-03 15:03:24,ModernCRM,nova.osapi_compute.wsgi.server [req-f2c16e44-d...,HTTP Status,bert
2060,2025-04-03 15:45:55,BillingSystem,nova.metadata.wsgi.server [req-d1ae2038-d065-4...,HTTP Status,bert


In [51]:
# install and import the correct package

from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['log_message'].tolist())
embeddings[:2]

array([[-1.02939628e-01,  3.35459672e-02, -2.20260601e-02,
         1.55105395e-03, -9.86921601e-03, -1.78956240e-01,
        -6.34409711e-02, -6.01761751e-02,  2.81109102e-02,
         5.99620305e-02, -1.72618218e-02,  1.43362966e-03,
        -1.49560004e-01,  3.15287290e-03, -5.66030778e-02,
         2.71685738e-02, -1.49890734e-02, -3.54037657e-02,
        -3.62936258e-02, -1.45410551e-02, -5.61493821e-03,
         8.75538960e-02,  4.55120802e-02,  2.50963848e-02,
         1.00187501e-02,  1.24266902e-02, -1.39923558e-01,
         7.68696517e-02,  3.14095318e-02, -4.15247073e-03,
         4.36902829e-02,  1.71250328e-02, -8.00951123e-02,
         5.74006140e-02,  1.89091992e-02,  8.55262727e-02,
         3.96398827e-02, -1.34371802e-01, -1.44364359e-03,
         3.06707504e-03,  1.76854074e-01,  4.44887020e-03,
        -1.69274695e-02,  2.24266481e-02, -4.35049385e-02,
         6.09032763e-03, -9.98167880e-03, -6.23972863e-02,
         1.07372468e-02, -6.04896527e-03, -7.14661106e-0

In [54]:
clusters = DBSCAN(eps=0.2, min_samples=1,metric='cosine').fit_predict(embeddings)
df['cluster'] = clusters
df[clusters == 1].head()

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1


In [43]:
clusters = df.groupby('cluster')['log_message'].apply(list)
sorted_clusters = clusters.sort_values(key=lambda x: x.map(len), ascending=False).head(10)


In [1]:
print("Clustered Patterns:")
for cluster_id, messages in sorted_clusters.items():
    if len(messages) > 10:
        print(f"Cluster {cluster_id}:")
        for msg in messages[:5]:
            print(f"  {msg}")

Clustered Patterns:


NameError: name 'sorted_clusters' is not defined

In [55]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            return label
    return None

In [ ]:
classify_with_regex("User User123 logged in.")

'User Action'

In [61]:
classify_with_regex("System reboot initiated by user User179.")

'System Notification'

In [63]:
classify_with_regex("Hey you, chill bro")


In [65]:
# Apply regex classification
df['regex_label'] = df['log_message'].apply(lambda x: classify_with_regex(x))
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [66]:
df[df['regex_label'].isnull()].head(5)

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [68]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 7)

In [70]:
df_legacy = df_non_regex[df_non_regex.source=="LegacyCRM"]
df_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,llm,24,None
255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,llm,48,None
377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,llm,62,None
1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,llm,105,None
1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,llm,118,None
1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,llm,122,None
2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,llm,133,None


In [72]:
df_non_legacy = df_non_regex[df_non_regex.source!="LegacyCRM"]
df_non_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


In [74]:
df_non_legacy.shape

(1903, 7)

In [77]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings_filtered = model.encode(df_non_legacy['log_message'].tolist())

In [78]:
len(embeddings_filtered)

1903

In [80]:
X = embeddings_filtered
y = df_non_legacy['target_label'].values

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [1]:
import os
import joblib

# Absolute path fix
BASE_DIR = os.path.dirname(os.getcwd())
MODEL_DIR = os.path.join(BASE_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, "log_classifier.joblib")

joblib.dump(clf, MODEL_PATH)

print("Model saved at:", MODEL_PATH)


NameError: name 'clf' is not defined